<a href="https://colab.research.google.com/github/Radhakrishna2055/GenerativeAI-B39/blob/main/GenAI_Assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
file_path = "Housing.csv"
df = pd.read_csv(file_path)

# Identify categorical and numerical columns
categorical_columns = ['mainroad', 'guestroom', 'basement', 'hotwaterheating',
                       'airconditioning', 'prefarea', 'furnishingstatus']
numerical_columns = [col for col in df.columns if col not in categorical_columns + ['price']]

# One-hot encode categorical variables
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Handle skewness in target variable
df['price'] = np.log1p(df['price'])

# Separate features and target variable
X = df.drop(columns=['price'])
y = df['price']

# Normalize numerical features
scaler = StandardScaler()
X[numerical_columns] = scaler.fit_transform(X[numerical_columns])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the improved ANN model
model = Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='mean_squared_error',
              metrics=['mae'])

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=32,
          validation_data=(X_test, y_test), verbose=1, callbacks=[early_stopping])

# Evaluate the model
train_loss, train_mae = model.evaluate(X_train, y_train, verbose=1)
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)
print(f'Training MAE: {train_mae}, Testing MAE: {test_mae}')

# Save the model in the recommended format
model.save("housing_price_model_optimized.keras")
print("Optimized model saved as 'housing_price_model_optimized.keras'.")

# Load and use the saved model
@tf.function(reduce_retracing=True)
def predict_model(input_data):
    return loaded_model(input_data, training=False)

loaded_model = tf.keras.models.load_model("housing_price_model_optimized.keras")
#The following line is changed to ensure the input data type is float32
input_sample = X_test[:5].astype(np.float32)
predictions = np.expm1(predict_model(input_sample))

# Display sample predictions
print("Sample Predictions:")
print(predictions)

Epoch 1/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 226.4056 - mae: 15.0307 - val_loss: 183.4930 - val_mae: 13.5306
Epoch 2/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 163.0880 - mae: 12.7080 - val_loss: 87.5780 - val_mae: 9.2252
Epoch 3/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 59.7220 - mae: 7.2296 - val_loss: 12.0195 - val_mae: 2.7016
Epoch 4/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 20.5955 - mae: 3.4868 - val_loss: 7.3518 - val_mae: 2.1039
Epoch 5/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 12.3429 - mae: 2.8087 - val_loss: 8.0808 - val_mae: 2.3201
Epoch 6/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 12.3084 - mae: 2.8176 - val_loss: 6.2558 - val_mae: 2.0230
Epoch 7/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 10.8282 - mae: 2.6129 - val_loss: 6.4473 - val_mae: 2.0744
Epoch 8/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 12.4510 - mae: 2.7939 - val_loss: 5.5914 - val_mae: 1.9366
Epoch 9/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/